In [153]:
import numpy as np
import pandas as pd

In [154]:
df = pd.read_csv('titled_players_raw.csv',index_col=0)

In [155]:
df.head()

,IDcode,Name,T,WT,Oth. T.,Fed,Rtg,Rpd,Blz,B-Year,S,F,Title
0,5813298,"Lim, Kok Ann",CM,,,SGP,,,,1920,M,,"['Candidate Master (CM)', '2012']"
1,4611870,"Keller-Hermann, Edith",WGM,WGM,,GER,2290,,,1921,F,i,"['Woman Grandmaster (WGM)', '1978', 'Woman Int..."
2,2002310,"Levy, Louis",FM,,,USA,2275,,,1921,M,i,[]
3,14100657,"Ousatchi, Mark",FM,,,GER,2157,,,1921,M,i,[]
4,4101529,"Averbakh, Yuri L",GM,,,RUS,2445,,,1922,M,i,"['Grandmaster (GM)', '1952', 'International Ma..."


In [156]:
df = df.replace({' ':np.nan,'[]': np.nan})

In [157]:
df.head()

,IDcode,Name,T,WT,Oth. T.,Fed,Rtg,Rpd,Blz,B-Year,S,F,Title
0,5813298,"Lim, Kok Ann",CM,NaN,NaN,SGP,NaN,NaN,NaN,1920,M,NaN,"['Candidate Master (CM)', '2012']"
1,4611870,"Keller-Hermann, Edith",WGM,WGM,NaN,GER,2290,NaN,NaN,1921,F,i,"['Woman Grandmaster (WGM)', '1978', 'Woman Int..."
2,2002310,"Levy, Louis",FM,NaN,NaN,USA,2275,NaN,NaN,1921,M,i,NaN
3,14100657,"Ousatchi, Mark",FM,NaN,NaN,GER,2157,NaN,NaN,1921,M,i,NaN
4,4101529,"Averbakh, Yuri L",GM,NaN,NaN,RUS,2445,NaN,NaN,1922,M,i,"['Grandmaster (GM)', '1952', 'International Ma..."


In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19510 entries, 0 to 19509
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   IDcode   19510 non-null  int64 
 1   Name     19510 non-null  object
 2   T        19510 non-null  object
 3   WT       3929 non-null   object
 4   Oth. T.  105 non-null    object
 5   Fed      19510 non-null  object
 6   Rtg      19436 non-null  object
 7   Rpd      11811 non-null  object
 8   Blz      12074 non-null  object
 9   B-Year   19508 non-null  object
 10  S        19510 non-null  object
 11  F        8260 non-null   object
 12  Title    15467 non-null  object
dtypes: int64(1), object(12)
memory usage: 2.1+ MB


La colonne Oth. T. est vide à 99% et nous donne des informations sur les joueurs ayant reçu des titres pour l'arbritrage. Cela ne nous intéresse pas.

Eliminons la

In [159]:
df = df.drop('Oth. T.', axis=1)

La colonne WT est vide à 79%

In [160]:
df['WT'].value_counts(normalize=True, dropna=False)

NaN     0.798616
 WFM    0.091748
 WIM    0.043875
 WCM    0.041312
 WGM    0.024398
 WH     0.000051
Name: WT, dtype: float64

Néanmoins 79% des titres sont détenus par les hommes

In [161]:
df['S'].value_counts(normalize=True)

M    0.797335
F    0.202665
Name: S, dtype: float64

Enlevons les espaces inutiles dans les colonnes de types object

In [162]:
cols = df.select_dtypes(include=['object']).columns
for col in cols:
    df[col] = df[col].str.strip()

In [163]:
df = df.rename(columns = {'IDcode':'id', 
           'Name':'nom',
           'T':'title',
           'WT':'woman_titre',
           'Fed':'pays',
           'Rtg':'classique',
           'Rpd':'rapide',
           'Blz':'blitz',
           'B-Year':'naissance',
           'S':'sexe',
           'F':'actif',
           'Title':'titre_dates'
          })

In [168]:
df[['classique', 'rapide', 'blitz', 'naissance']] = df[['classique', 'rapide', 'blitz', 'naissance']].astype('float')

Remplaçons la colonne actif par 1 si le joueur est actif 0 sinon

In [171]:
df['actif'].value_counts()

i    8260
Name: actif, dtype: int64

A vérifier si l'in/activité des joueurs semble correct

In [172]:
# df['actif'] = df['actif']
df['actif'].replace({'i':'0',np.nan:'1'})

0        1
1        0
2        0
3        0
4        0
        ..
19505    1
19506    1
19507    1
19508    1
19509    1
Name: actif, Length: 19510, dtype: object

In [56]:
df.iloc[df['Title'].apply(lambda x:len(x)).sort_values(ascending=False)[:100].index]

17075    ['Grandmaster (GM)', '2021', 'International Ma...
16439    ['International Master (IM)', '2016', 'Woman G...
17179    ['International Master (IM)', '2019', 'FIDE Ma...
15924    ['International Master (IM)', '2017', 'Woman G...
14207    ['International Master (IM)', '2017', 'Woman G...
                               ...                        
11091    ['FIDE Master (FM)', '2017', 'Title', 'FIDE PB...
15176    ['Woman Grandmaster (WGM)', '2017', 'Woman Int...
11898    ['FIDE Master (FM)', '2005', 'Title', 'FIDE PB...
16305    ['Woman Grandmaster (WGM)', '2019', 'Woman Int...
16618    ['Grandmaster (GM)', '2016', 'International Ma...
Name: Title, Length: 100, dtype: object


In [58]:
df['Title'].iloc[17075]

"['Grandmaster (GM)', '2021', 'International Master (IM)', '2017', 'Woman Grandmaster (WGM)', '2014', 'Woman International Master (WIM)', '2011', 'Woman FIDE Master (WFM)', '2010', 'Title', 'FIDE PB / Congress', 'Status', 'Grandmaster', '89th FIDE Congress 2018, 26 Sep - 6 Oct, Batumi, Georgia', 'YES', 'International Master', 'Title Applications approved by the Presidential Board by written resolution', 'YES', 'Woman Grandmaster', '1st quarter Presidential Board Meeting, Khanty-Mansiysk, RUS, 29 March - 1 April 2014', 'YES']"

In [62]:
df.iloc[[17075]]

,IDcode,Name,T,WT,Oth. T.,Fed,Rtg,Rpd,Blz,B-Year,S,F,Title
17075,13703544,"Abdumalik, Zhansaya",GM,WGM,,KAZ,2502,2444,2380,2000,F,,"['Grandmaster (GM)', '2021', 'International Ma..."
